read data

In [1]:
import os
import pandas as pd
from tqdm import tqdm
from urllib.parse import urlparse
data_list = []
for home, dirs, files in os.walk("数据集/monkey数据集/数据集/"):
    for filename in files:
        fullname = os.path.join(home, filename)
        file = open(fullname, "r")
        try:
            content = file.read()
            file.close()
            data_list.append((fullname, content))
            file.close()
        except:
            file.close()
            pass
data = pd.DataFrame(data_list, columns=['path', 'text'])

read index

In [2]:
index = pd.read_csv("数据集/monkey数据集/数据集说明/index", header=None)
index.columns = ["index"]
index['path'] = index['index'].apply(lambda x: "数据集/monkey数据集/数据集" + x.split("data")[1])
index['nor_phish'] = index['index'].apply(lambda x: x.split(" ")[0])
index['label'] = index['nor_phish'].apply(lambda x: 1 if x == "phishing" else 0)
# index

FileNotFoundError: [Errno 2] No such file or directory: '数据集/monkey数据集/数据集说明/index'

merge data and label

In [ ]:
data = pd.merge(data, index, on="path", how="inner").iloc[:, [1, 4]].drop_duplicates()

In [ ]:
data.groupby("label").count()

extract body

In [ ]:
def extract_body(text):
    import email.parser
    import email
    from email.parser import Parser
    body = ''
    subject = ''
    date = ''
    parser = Parser()
    msg = email.message_from_string(text)
    email = parser.parsestr(text)
    if msg.is_multipart():
        for payload in msg.get_payload():
            body += str(payload)
        try:
            for sub in msg.get('subject'):
                subject += str(sub)
        except:
            subject = ''
    else:
        body = msg.get_payload()
        try:
            subject = msg.get("subject")
        except:
            subject = ''
        date = msg.get("Date")
    return body, subject, date
data['body'] = data['text'].apply(lambda x: extract_body(x)[0])
data['subject'] = data['text'].apply(lambda x: extract_body(x)[1])
data['Date'] = data['text'].apply(lambda x: extract_body(x)[2])

extract url from body

In [ ]:
from urlextract import URLExtract
extractor = URLExtract()
data['urls'] = data['body'].apply(lambda x:list(set(extractor.find_urls(x))))
# 判断发件时间是否为非工作日
def judge_email_time(x):
    if x:
        if "Sat" in x or "Sun" in x:
            return 1
        else:
            return 0
    else:
        return 0
data['date_work'] = data['Date'].apply(judge_email_time)
data

feature engineering-url

In [ ]:
# 1. count of dot
from collections import Counter
def url_to_domain(url):
	o = urlparse(url)
	domain = o.hostname
	return domain
data['dot'] = data['urls'].apply(lambda xs: sum([Counter(url_to_domain(x))['.'] for x in xs]))

In [ ]:
# 5. 是否含有端口号
from urllib.parse import urlparse
def ifcontainport(urls):
    count = 0
    for url in urls:
        if urlparse(url).port:
            count += 1
    if count > 0:
        return 1
    else:
        return 0
data['port'] = data['urls'].apply(lambda xs: 1 if list(set([urlparse(x).port for x in xs]))!=[None] and len(xs)>0 else 0)
data

In [ ]:
# 6. 含有端口号的url个数
from urllib.parse import urlparse
def count_urls_contain_post(urls):
    count = 0
    for url in urls:
        if urlparse(url).port:
            count += 1
    return count
    
data['port1'] = data['urls'].apply(lambda x: count_urls_contain_post(x))

In [ ]:
# 8. 包含ip的url个数
import re
def containipinlink(urls):
    pattern = re.compile(r'(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})')
    count = 0
    for url in urls:
        try:
            if pattern.search(url)[0]:
                count += 1
            else:
                pass
        except:
            pass
    return count
data['ip'] = data['urls'].apply(lambda x: containipinlink(x))

In [ ]:
# 9. url 使用ip地址
data['ip1'] = data['ip'].apply(lambda x: 1 if x>0 else 0)

In [ ]:
# 10. 连续使用域名数量
data['domain_count'] = data['urls'].apply(lambda xs: len(set([url_to_domain(x) for x in xs])))

In [ ]:
# 11.链接数量
data['urls_count'] = data['urls'].apply(lambda xs: len(xs))
# 13.14.click here link
data['click'] = data['urls'].apply(lambda x: 1 if 'click' in ",".join(x).lower() else 0)
data['here'] = data['urls'].apply(lambda x: 1 if 'here' in ",".join(x).lower() else 0)
data['link'] = data['urls'].apply(lambda x: 1 if 'link' in ",".join(x).lower() else 0)

feature engineering-body

In [ ]:
# 1.suspension
data['suspension'] = data['body'].apply(lambda x: 1 if "suspension" in x.lower() else 0)

In [ ]:
# 2.dear
data['dear'] = data['body'].apply(lambda x: 1 if "dear" in x.lower() else 0)

In [ ]:
# 3. verify your account
data['verifyyouraccount'] = data['body'].apply(lambda x: 1 if "verify your account" in x.lower() else 0)

In [ ]:
# 4. HTML
data['html'] = data['body'].apply(lambda x: 1 if "html" in x.lower() else 0)

In [ ]:
# 7. 邮件正文丰富度 = 词的数量/字符的数量。
data['w_count'] = data['body'].apply(lambda x: len(x.split(" ")))
data['str_count'] = data['body'].apply(lambda x: len(x))
data['body_richness'] = round(data['w_count'] / data['str_count'], 2)

In [ ]:
# 8.account
data['account_count'] = data['body'].apply(lambda x: Counter(x.split(" "))['account'])

In [ ]:
# 9. suspended
data['suspended_count'] = data['body'].apply(lambda x: Counter(x.split(" "))['suspended'])

In [ ]:
# 10. 不重复单词个数
data['word_count'] = data['body'].apply(lambda x: len(set(x.split(" "))))

In [ ]:
# 特殊字符
# @：在电子邮件地址中使用，用于分隔用户名和域名。
data['@'] = data['body'].apply(lambda x: 1 if "@" in x.lower() else 0)
# !：用于增加紧迫感或强调重要性。
data['!'] = data['body'].apply(lambda x: 1 if "!" in x.lower() else 0)
# # ：用于标记话题或关键字。
data['#'] = data['body'].apply(lambda x: 1 if "#" in x.lower() else 0)
# %：用于表示折扣或优惠。
data['%'] = data['body'].apply(lambda x: 1 if "%" in x.lower() else 0)
# &：用于连接两个词或短语。
data['&'] = data['body'].apply(lambda x: 1 if "&" in x.lower() else 0)
# +：用于创建假电子邮件地址。
data['+'] = data['body'].apply(lambda x: 1 if "+" in x.lower() else 0)
# -：用于创建假网站地址或分隔单词。
data['-'] = data['body'].apply(lambda x: 1 if "-" in x.lower() else 0)
# _：用于创建假电子邮件地址。
data['_'] = data['body'].apply(lambda x: 1 if "_" in x.lower() else 0)
# $：用于表示货币或价格。
data['$'] = data['body'].apply(lambda x: 1 if "$" in x.lower() else 0)
# *：用于强调或突出显示重要部分。
data['*'] = data['body'].apply(lambda x: 1 if "*" in x.lower() else 0)
# /：用于创建假网站地址。
data['/'] = data['body'].apply(lambda x: 1 if "/" in x.lower() else 0)
# \：用于混淆网址或文件路径
data['\\'] = data['body'].apply(lambda x: 1 if "\\" in x.lower() else 0)

In [ ]:
# 钓鱼邮件中敏感词
data['contain_account'] = data['body'].apply(lambda x: 1 if "account" in x.lower() else 0)
data['contain_password'] = data['body'].apply(lambda x: 1 if "password" in x.lower() else 0)
data['contain_verify'] = data['body'].apply(lambda x: 1 if "verify" in x.lower() else 0)
data['contain_reward'] = data['body'].apply(lambda x: 1 if "reward" in x.lower() else 0)
data['contain_confidentiality'] = data['body'].apply(lambda x: 1 if "confidentiality" in x.lower() else 0)
data['contain_cancel'] = data['body'].apply(lambda x: 1 if "cancel" in x.lower() else 0)

feature engineering-subject

In [ ]:
# 1.bank
data['bank'] = data['subject'].apply(lambda x: 1 if x and 'bank' in x.lower() else 0)

In [ ]:
# 3. FW
data['fw'] = data['subject'].apply(lambda x: 1 if x and 'FW' in x else 0)

In [ ]:
# 4. re
data['re'] = data['subject'].apply(lambda x: 1 if x and 're:' in x else 0)

In [ ]:
# 5. verify
data['verify'] = data['subject'].apply(lambda x: 1 if x and 'verify' in x else 0)

In [ ]:
# 6.word count
data['subject_word_count'] = data['subject'].apply(lambda x: len(str(x).split(" ")) if x else 0)

In [ ]:
# 7.str count
data['subject_str_count'] = data['subject'].apply(lambda x: len(str(x)) if x else 0)

In [ ]:
# richness
data['subject_richness'] = round(data['subject_word_count'] / data['subject_str_count'], 2)

In [ ]:
# 是否包含大写字母
data['upper'] = data['subject'].apply(lambda x: 0 if x and x.islower() else 1)

In [ ]:
import numpy as np
data = data.fillna(0)
data = data.replace([np.inf, -np.inf], 0)

In [ ]:
data.groupby('label').count()

In [ ]:
data

In [ ]:
import numpy as np
print(np.isnan(data.iloc[:, 6:]).any())

In [ ]:
features = data.iloc[:, 6:]
labels = data.iloc[:, [1]]
for i in list(features.columns):
   # 获取各个指标的最大值和最小值
    Max = np.max(features[i])
    Min = np.min(features[i])
    features[i] = (features[i] - Min)/(Max - Min)
features = features.fillna(0)

In [ ]:
pd.DataFrame(labels).groupby("label").count()

split train and test data

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data, train_label, test_label = train_test_split(features, labels, test_size=0.2, random_state=1)

In [ ]:
test_data

model

RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=0)
rfc = rfc.fit(train_data, train_label)
predict = rfc.predict(test_data)
from sklearn.metrics import classification_report
print(classification_report(test_label, predict))
from sklearn.metrics import confusion_matrix
c_m = confusion_matrix(test_label, predict)
print(c_m)
import seaborn as sns
sns.heatmap(c_m)
rfc.predict_proba(test_data)

decision tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(criterion='gini', splitter='best',max_depth=32,min_samples_split=2)
dt = dt.fit(train_data, train_label)
predict = dt.predict(test_data)
from sklearn.metrics import classification_report
print(classification_report(test_label, predict))
from sklearn.metrics import confusion_matrix
c_m = confusion_matrix(test_label, predict)
print(c_m)
import seaborn as sns
sns.heatmap(c_m)
dt.predict_proba(test_data)

朴素贝叶斯

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb = gnb.fit(train_data, train_label)
predict = gnb.predict(test_data)
from sklearn.metrics import classification_report
print(classification_report(test_label, predict))
from sklearn.metrics import confusion_matrix
c_m = confusion_matrix(test_label, predict)
print(c_m)
import seaborn as sns
sns.heatmap(c_m)
gnb.predict_proba(test_data)

神经网络

In [ ]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5,2), random_state=1)  # 神经网络输入为2，第一隐藏层神经元个数为5，第二隐藏层神经元个数为2，输出结果为2分类。
clf = clf.fit(train_data, train_label)
predict = clf.predict(test_data)
from sklearn.metrics import classification_report
print(classification_report(test_label, predict))
from sklearn.metrics import confusion_matrix
c_m = confusion_matrix(test_label, predict)
print(c_m)
import seaborn as sns
sns.heatmap(c_m)
clf.predict_proba(test_data)



SVM

In [ ]:
from sklearn import svm
svc = svm.SVC(kernel="rbf", C=1, probability=True)
svc = svc.fit(train_data, train_label)
predict = svc.predict(test_data)
from sklearn.metrics import classification_report
print(classification_report(test_label, predict))
from sklearn.metrics import confusion_matrix
c_m = confusion_matrix(test_label, predict)
print(c_m)
import seaborn as sns
sns.heatmap(c_m)
svc.predict_proba(test_data)